<a href="https://colab.research.google.com/github/sammainahkinya1404/A-novel-detection-method-based-on-DETR-for-drone-aerial-images/blob/main/Drone_Aerial_Images_Detection_Using_DETR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import json
import urllib.request
from zipfile import ZipFile
from pathlib import Path


In [5]:
# function to downlaod from github extract images from zip file
def download_extract_images(urls, extract_dir):
    os.makedirs(extract_dir, exist_ok=True)
    for url in urls:
        file_name = url.split('/')[-1]
        file_path = os.path.join(extract_dir, file_name)
        urllib.request.urlretrieve(url, file_path)
        with ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        os.remove(file_path)

In [6]:
# Function to convert annotations of the Images into COCO format
def convert_annotations_to_coco_format(annotation_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for file_name in os.listdir(annotation_dir):
        if file_name.endswith('.txt'):
            annotations = []
            image_id = 0
            with open(os.path.join(annotation_dir, file_name), 'r') as f:
                for line in f.readlines():
                    values = line.strip().split()
                    if len(values) > 0:
                        category_id = int(values[0])
                        bbox = list(map(float, values[1:]))
                        annotations.append({
                            "id": len(annotations),
                            "image_id": image_id,
                            "category_id": category_id,
                            "bbox": bbox,
                            "area": bbox[2] * bbox[3],
                            "iscrowd": 0
                        })
            coco_format = {
                "images": [{"id": 0, "file_name": file_name.replace('.txt', '.jpg')}],
                "annotations": annotations,
                "categories": [{"id": 0, "name": "object"}]
            }
            output_file_path = os.path.join(output_dir, file_name.replace('.txt', '.json'))
            with open(output_file_path, 'w') as f:
                json.dump(coco_format, f)

In [8]:
# Define paths to store Data Images
dataset_root = Path('/content/drive/MyDrive/Colab Notebooks/')
download_dir = dataset_root / "downloaded_images"
annotation_dir = dataset_root / "annotations"
output_dir = dataset_root / "coco_annotations"


In [12]:
annotation_dir = dataset_root / "annotations"
os.makedirs(annotation_dir, exist_ok=True)  # Ensure the annotations directory exists
output_dir = dataset_root / "coco_annotations"

In [9]:
# Define URLs for the data
urls = [
    'https://github.com/ultralytics/yolov5/releases/download/v1.0/VisDrone2019-DET-train.zip',
    'https://github.com/ultralytics/yolov5/releases/download/v1.0/VisDrone2019-DET-val.zip',
    'https://github.com/ultralytics/yolov5/releases/download/v1.0/VisDrone2019-DET-test-dev.zip',
    'https://github.com/ultralytics/yolov5/releases/download/v1.0/VisDrone2019-DET-test-challenge.zip'
]


In [10]:
# Download and extract images
download_extract_images(urls, download_dir)

In [13]:



# Convert annotations to COCO format
convert_annotations_to_coco_format(annotation_dir, output_dir)


The Tasks Was Trained using The Orignal DETR model,which is What am Going to do,here is the Bulding Blocks of the Original DETR:


**1.ResNet-50 Backbone**



**2.Transformer Encoder**


**3.Transformer Decoder**

**4.Prediction Feed Forward Network[FFN]**

In [14]:
# Install all Libraries Needed
!pip install -q datasets transformers evaluate timm albumentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00


In [15]:
from datasets import load_dataset
